### The script below syncs the supply Chain BOM with Engineering BOM.
### Created by Yifei.li@byton.com

In [4]:
import pandas as pd
import numpy as np
import xlrd, openpyxl
import csv, datetime
import time

# put up-to-date EBOM and SCBOM here for sync
EBOM_PATH = './Data/EBOM_10.8.18_addQuan.xlsx'
SCBOM_PATH = './Data/Copy of Supply Chain BOM_2018-10-03_final.xlsx'


# needs to change the followings based on how the EBOM and SCBOM columns are structured 
# 0 index
SCBOM_columns_start = 14 # SC columns start at column # 15
SCBOM_columns_end = 78 # SC columns end at column # 77
SCBOM_updated_columns_start = 14 
SCBOM_updated_columns_end = 78

# tab names in supply chain BOM.xlsx and EBOM.xlsx
system_name = ["A BIW", "B Closures", "C Exterior", "D Interior", "E Chassis", "F Thermal Management", "G Drivetrain",
              "H Power Electronics", "J HV Battery", "K Autonomy", "L Low Voltage Systems", "M Connectivity", 
              "N Intelligent Car Experience ICE", "X Raw Materials", "Y Fasteners", "Z Vehicle Top Level Cfg"]

In [22]:
#load EBOM and SCBOM
def load():
    # load Engineering BOM
    # EBOM = pd.read_csv(EBOM_PATH)
    EBOM = pd.read_excel(EBOM_PATH, sheet_name="BOM")
    # EBOM.columns = EBOM.columns.str.replace("\(R\)\ ", "") # trim (R) away from the header
    EBOM.loc[EBOM.System=="N Intelligent Car Experience ICE", "System"]= "N ICE"
    EBOM = EBOM.reset_index(drop=True)

#     # load Supply Chain BOM from MULTIPLE tabs in an Excel
#     df= pd.read_excel(SCBOM_PATH, sheet_name=system_name)
#     SCBOM = pd.DataFrame()
#     for each in df:
#         SCBOM = SCBOM.append(df[each])

    # load Supply Chain BOM from SINGLE TAB in an Excel
    SCBOM= pd.read_excel(SCBOM_PATH, sheet_name="Supply Chain BOM")
    
    SCBOM.loc[SCBOM.System=="N Intelligent Car Experience ICE", "System"]= "N ICE"
    # reset index of SCBOM
    SCBOM.reset_index(drop=True, inplace=True)
    print("Completed loading excel files\n")
    return EBOM, SCBOM

In [43]:
EBOM, SCBOM = load()
SCBOM.loc[SCBOM.System=="N Intelligent Car Experience ICE", "System"]= "N ICE"

# SCBOM[SCBOM["System"]=="N ICE"]
SCBOM.loc[SCBOM.System=="N ICE", "System"]

Completed loading excel files



3639    N ICE
3640    N ICE
3641    N ICE
3642    N ICE
3643    N ICE
3644    N ICE
3645    N ICE
3646    N ICE
3647    N ICE
3648    N ICE
3649    N ICE
3650    N ICE
3651    N ICE
3652    N ICE
3653    N ICE
3654    N ICE
3655    N ICE
3656    N ICE
3657    N ICE
3658    N ICE
3659    N ICE
3660    N ICE
3661    N ICE
3662    N ICE
3663    N ICE
3664    N ICE
3665    N ICE
3666    N ICE
3667    N ICE
3668    N ICE
3669    N ICE
3670    N ICE
3671    N ICE
3672    N ICE
3673    N ICE
3674    N ICE
3675    N ICE
3676    N ICE
3677    N ICE
3678    N ICE
3679    N ICE
3680    N ICE
3681    N ICE
3682    N ICE
3683    N ICE
3684    N ICE
3685    N ICE
3686    N ICE
3687    N ICE
3688    N ICE
3689    N ICE
3690    N ICE
3691    N ICE
3692    N ICE
3693    N ICE
3694    N ICE
Name: System, dtype: object

In [4]:
# search PN and/or Rev in SCBOM
def search(df, PN, Rev):
    # df = df.loc[(df["Title"]==PN) & (df["Revision"]==Rev)]
    df = df.loc[(df["Title"]==PN)]	# search PN only 
    index = df.index.values
    
    # only one entry found
    if (len(index)==1):
        return index   
    
    # multiple entries found
    elif (len(index)>1):
        #return the first index if duplicate PN and Rev found
        return index.tolist()   # returned indexes are a numpy.ndarray, we convert to a list and get the first value
    
    # no entry found
    else:
        return None

In [5]:
# copy SCBOM info and paste into SCBOM_updated
def copy_and_paste_row(df1, index1, df2, index2):  # index1 and index2 are int
    # copy df2 info into df1
    # only copy columns that are not on EBOM
	df1.loc[index1, df1.columns.tolist()[SCBOM_updated_columns_start:SCBOM_updated_columns_end]] = df2.loc[index2, df2.columns.tolist()[SCBOM_columns_start:SCBOM_columns_end]]
	return df1

In [6]:
# save dataframe to Excel
def save(df):
	#https://stackoverflow.com/questions/28837057/pandas-writing-an-excel-file-containing-unicode-illegalcharactererror
	df = df.applymap(lambda x: x.encode('unicode_escape').
	             decode('utf-8') if isinstance(x, str) else x)
	date = str(datetime.date.today())
	name = "Supply Chain BOM_" + date + ".xlsx"
	writer = pd.ExcelWriter(name)

	# saving to multiple tabs
	for each in system_name:
		system = df[df["System"]==each]
		print("saving: {}".format(each))
		system.to_excel(writer, sheet_name=each, na_rep="")
	

	# # saving to ONE tab
	# df.to_excel(writer, sheet_name="Updated Supply Chain BOM", na_rep="" )

	writer.save()

#     df.to_csv("Supply Chain BOM.csv",quoting=csv.QUOTE_NONE, escapechar="\\")

In [7]:
start_time = time.time()
EBOM, SCBOM = load()

# create a updated SCBOM with columns from SCBOM and data from EBOM
SCBOM_updated = EBOM.copy()
# copy only colums that do not exist in EBOM
for each in SCBOM.columns.tolist()[SCBOM_columns_start:]:
    SCBOM_updated[each] = ""


SCBOM_columns = SCBOM.columns.tolist()
SCBOM_columns_size = len(SCBOM_columns)
EBOM_columns = EBOM.columns.tolist()
EBOM_columns_size = len(EBOM_columns)

print("Before Sync")
print("EBOM shape: ", EBOM.shape)
print("SCBOM shape: ", SCBOM.shape)
print("SCBOM_updated ", SCBOM_updated.shape)


# count how many new parts are added
# how many old parts are removed
removed_parts_count = 0
same_parts = 0


# loop through SCBOM 
for index, row in SCBOM.iterrows():
    print("iterations: {}\t ".format(index))
    PN = row["Title"]
    Rev = row["Revision"]
    # same part could be structured differently but the part is the same, 
    # so if same part found at multiple places, we will just pick one
    # Identifier = row["Identifier"] 

    # search if this part exists in SCBOM_updated, return the index in SCBOM_updated
    # search() returned a list or None type
    index_SCBOM_updated = search(SCBOM_updated, PN, Rev)

    #not found, deactivate the part, then append this part to SCBOM_updated
    if (index_SCBOM_updated == None):
        SCBOM.loc[index, ["Part Active"]] = "Inactivate"
        SCBOM.loc[index, ["Part Status"]] = "Removed"
        # SCBOM.loc[index, ["Part Creation Date"]] = datetime.date(2019, 9, 14)
        SCBOM.loc[index, ["Last Modified Date"]] = datetime.date.today()
        # SCBOM.loc[index, ["Last Modified Date"]] = datetime.date(2019, 9, 28)
        columns_to_copy = SCBOM.columns.tolist()[SCBOM_columns_start:SCBOM_columns_end]
        other_columns = ["Title", "Revision", "Description", "System", "SubSystem", "Part Type"]
        columns_to_copy = columns_to_copy + other_columns
        SCBOM_updated = SCBOM_updated.append(SCBOM.loc[index, columns_to_copy], ignore_index=True)

        removed_parts_count = removed_parts_count + 1

    # found one entry, copy the information to updated SCBOM    
    elif (len(index_SCBOM_updated) == 1):
        index_SCBOM_updated = index_SCBOM_updated[0]
        SCBOM_updated = copy_and_paste_row(SCBOM_updated, index_SCBOM_updated, SCBOM, index)
        SCBOM_updated.loc[index_SCBOM_updated, ["Part Creation Date"]] = datetime.date(2019, 9, 21)
        # SCBOM_updated.loc[index_SCBOM_updated, ["Last Modified Date"]] = datetime.date.today()
        SCBOM_updated.loc[index_SCBOM_updated, ["Part Status"]] = "Lateste Revision"
        SCBOM_updated.loc[index_SCBOM_updated, ["Part Active"]] = "Active"
        same_parts = same_parts + 1

    # found multiple entries, copy the information to each of the entries in SCBOM_updated
    else:
        for each in index_SCBOM_updated:
            SCBOM_updated = copy_and_paste_row(SCBOM_updated, each, SCBOM, index)
            SCBOM_updated.loc[index_SCBOM_updated, ["Part Creation Date"]] = datetime.date(2019, 9, 21)
            # SCBOM_updated.loc[index_SCBOM_updated, ["Last Modified Date"]] = datetime.date.today()
            SCBOM_updated.loc[index_SCBOM_updated, ["Part Status"]] = "Lateste Revision"
            SCBOM_updated.loc[index_SCBOM_updated, ["Part Active"]] = "Active"
        same_parts = same_parts + 1

save(SCBOM_updated)

print("\nAfter Sync")
print("EBOM shape: ", EBOM.shape)
print("SCBOM shape: ", SCBOM.shape)
print("SCBOM_updated ", SCBOM_updated.shape)


print("\n# of new parts added: ", EBOM.shape[0]-same_parts)
print("# of old parts removed: ", removed_parts_count)
print("# of additional columns added in EBOM: ", SCBOM_updated.shape[1] - SCBOM.shape[1])


execution_time = round(time.time() - start_time, 2)
print("\nThis script took--- {} seconds ---".format(execution_time))


Completed loading excel files

Before Sync
EBOM shape:  (4671, 14)
SCBOM shape:  (4994, 78)
SCBOM_updated  (4671, 78)
iterations: 0	 
iterations: 1	 
iterations: 2	 
iterations: 3	 
iterations: 4	 
iterations: 5	 
iterations: 6	 
iterations: 7	 
iterations: 8	 
iterations: 9	 
iterations: 10	 
iterations: 11	 
iterations: 12	 
iterations: 13	 
iterations: 14	 
iterations: 15	 
iterations: 16	 
iterations: 17	 
iterations: 18	 
iterations: 19	 
iterations: 20	 
iterations: 21	 
iterations: 22	 
iterations: 23	 
iterations: 24	 
iterations: 25	 
iterations: 26	 
iterations: 27	 
iterations: 28	 
iterations: 29	 
iterations: 30	 
iterations: 31	 
iterations: 32	 
iterations: 33	 
iterations: 34	 
iterations: 35	 
iterations: 36	 
iterations: 37	 
iterations: 38	 
iterations: 39	 
iterations: 40	 
iterations: 41	 
iterations: 42	 
iterations: 43	 
iterations: 44	 
iterations: 45	 
iterations: 46	 
iterations: 47	 
iterations: 48	 
iterations: 49	 
iterations: 50	 
iterations: 51	 
iteratio

iterations: 456	 
iterations: 457	 
iterations: 458	 
iterations: 459	 
iterations: 460	 
iterations: 461	 
iterations: 462	 
iterations: 463	 
iterations: 464	 
iterations: 465	 
iterations: 466	 
iterations: 467	 
iterations: 468	 
iterations: 469	 
iterations: 470	 
iterations: 471	 
iterations: 472	 
iterations: 473	 
iterations: 474	 
iterations: 475	 
iterations: 476	 
iterations: 477	 
iterations: 478	 
iterations: 479	 
iterations: 480	 
iterations: 481	 
iterations: 482	 
iterations: 483	 
iterations: 484	 
iterations: 485	 
iterations: 486	 
iterations: 487	 
iterations: 488	 
iterations: 489	 
iterations: 490	 
iterations: 491	 
iterations: 492	 
iterations: 493	 
iterations: 494	 
iterations: 495	 
iterations: 496	 
iterations: 497	 
iterations: 498	 
iterations: 499	 
iterations: 500	 
iterations: 501	 
iterations: 502	 
iterations: 503	 
iterations: 504	 
iterations: 505	 
iterations: 506	 
iterations: 507	 
iterations: 508	 
iterations: 509	 
iterations: 510	 
iterations

iterations: 912	 
iterations: 913	 
iterations: 914	 
iterations: 915	 
iterations: 916	 
iterations: 917	 
iterations: 918	 
iterations: 919	 
iterations: 920	 
iterations: 921	 
iterations: 922	 
iterations: 923	 
iterations: 924	 
iterations: 925	 
iterations: 926	 
iterations: 927	 
iterations: 928	 
iterations: 929	 
iterations: 930	 
iterations: 931	 
iterations: 932	 
iterations: 933	 
iterations: 934	 
iterations: 935	 
iterations: 936	 
iterations: 937	 
iterations: 938	 
iterations: 939	 
iterations: 940	 
iterations: 941	 
iterations: 942	 
iterations: 943	 
iterations: 944	 
iterations: 945	 
iterations: 946	 
iterations: 947	 
iterations: 948	 
iterations: 949	 
iterations: 950	 
iterations: 951	 
iterations: 952	 
iterations: 953	 
iterations: 954	 
iterations: 955	 
iterations: 956	 
iterations: 957	 
iterations: 958	 
iterations: 959	 
iterations: 960	 
iterations: 961	 
iterations: 962	 
iterations: 963	 
iterations: 964	 
iterations: 965	 
iterations: 966	 
iterations

iterations: 1349	 
iterations: 1350	 
iterations: 1351	 
iterations: 1352	 
iterations: 1353	 
iterations: 1354	 
iterations: 1355	 
iterations: 1356	 
iterations: 1357	 
iterations: 1358	 
iterations: 1359	 
iterations: 1360	 
iterations: 1361	 
iterations: 1362	 
iterations: 1363	 
iterations: 1364	 
iterations: 1365	 
iterations: 1366	 
iterations: 1367	 
iterations: 1368	 
iterations: 1369	 
iterations: 1370	 
iterations: 1371	 
iterations: 1372	 
iterations: 1373	 
iterations: 1374	 
iterations: 1375	 
iterations: 1376	 
iterations: 1377	 
iterations: 1378	 
iterations: 1379	 
iterations: 1380	 
iterations: 1381	 
iterations: 1382	 
iterations: 1383	 
iterations: 1384	 
iterations: 1385	 
iterations: 1386	 
iterations: 1387	 
iterations: 1388	 
iterations: 1389	 
iterations: 1390	 
iterations: 1391	 
iterations: 1392	 
iterations: 1393	 
iterations: 1394	 
iterations: 1395	 
iterations: 1396	 
iterations: 1397	 
iterations: 1398	 
iterations: 1399	 
iterations: 1400	 
iterations: 

iterations: 1781	 
iterations: 1782	 
iterations: 1783	 
iterations: 1784	 
iterations: 1785	 
iterations: 1786	 
iterations: 1787	 
iterations: 1788	 
iterations: 1789	 
iterations: 1790	 
iterations: 1791	 
iterations: 1792	 
iterations: 1793	 
iterations: 1794	 
iterations: 1795	 
iterations: 1796	 
iterations: 1797	 
iterations: 1798	 
iterations: 1799	 
iterations: 1800	 
iterations: 1801	 
iterations: 1802	 
iterations: 1803	 
iterations: 1804	 
iterations: 1805	 
iterations: 1806	 
iterations: 1807	 
iterations: 1808	 
iterations: 1809	 
iterations: 1810	 
iterations: 1811	 
iterations: 1812	 
iterations: 1813	 
iterations: 1814	 
iterations: 1815	 
iterations: 1816	 
iterations: 1817	 
iterations: 1818	 
iterations: 1819	 
iterations: 1820	 
iterations: 1821	 
iterations: 1822	 
iterations: 1823	 
iterations: 1824	 
iterations: 1825	 
iterations: 1826	 
iterations: 1827	 
iterations: 1828	 
iterations: 1829	 
iterations: 1830	 
iterations: 1831	 
iterations: 1832	 
iterations: 

...

iterations: 3948	 
iterations: 3949	 
iterations: 3950	 
iterations: 3951	 
iterations: 3952	 
iterations: 3953	 
iterations: 3954	 
iterations: 3955	 
iterations: 3956	 
iterations: 3957	 
iterations: 3958	 
iterations: 3959	 
iterations: 3960	 
iterations: 3961	 
iterations: 3962	 
iterations: 3963	 
iterations: 3964	 
iterations: 3965	 
iterations: 3966	 
iterations: 3967	 
iterations: 3968	 
iterations: 3969	 
iterations: 3970	 
iterations: 3971	 
iterations: 3972	 
iterations: 3973	 
iterations: 3974	 
iterations: 3975	 
iterations: 3976	 
iterations: 3977	 
iterations: 3978	 
iterations: 3979	 
iterations: 3980	 
iterations: 3981	 
iterations: 3982	 
iterations: 3983	 
iterations: 3984	 
iterations: 3985	 
iterations: 3986	 
iterations: 3987	 
iterations: 3988	 
iterations: 3989	 
iterations: 3990	 
iterations: 3991	 
iterations: 3992	 
iterations: 3993	 
iterations: 3994	 
iterations: 3995	 
iterations: 3996	 
iterations: 3997	 
iterations: 3998	 
iterations: 3999	 
iterations: 

iterations: 4380	 
iterations: 4381	 
iterations: 4382	 
iterations: 4383	 
iterations: 4384	 
iterations: 4385	 
iterations: 4386	 
iterations: 4387	 
iterations: 4388	 
iterations: 4389	 
iterations: 4390	 
iterations: 4391	 
iterations: 4392	 
iterations: 4393	 
iterations: 4394	 
iterations: 4395	 
iterations: 4396	 
iterations: 4397	 
iterations: 4398	 
iterations: 4399	 
iterations: 4400	 
iterations: 4401	 
iterations: 4402	 
iterations: 4403	 
iterations: 4404	 
iterations: 4405	 
iterations: 4406	 
iterations: 4407	 
iterations: 4408	 
iterations: 4409	 
iterations: 4410	 
iterations: 4411	 
iterations: 4412	 
iterations: 4413	 
iterations: 4414	 
iterations: 4415	 
iterations: 4416	 
iterations: 4417	 
iterations: 4418	 
iterations: 4419	 
iterations: 4420	 
iterations: 4421	 
iterations: 4422	 
iterations: 4423	 
iterations: 4424	 
iterations: 4425	 
iterations: 4426	 
iterations: 4427	 
iterations: 4428	 
iterations: 4429	 
iterations: 4430	 
iterations: 4431	 
iterations: 

iterations: 4812	 
iterations: 4813	 
iterations: 4814	 
iterations: 4815	 
iterations: 4816	 
iterations: 4817	 
iterations: 4818	 
iterations: 4819	 
iterations: 4820	 
iterations: 4821	 
iterations: 4822	 
iterations: 4823	 
iterations: 4824	 
iterations: 4825	 
iterations: 4826	 
iterations: 4827	 
iterations: 4828	 
iterations: 4829	 
iterations: 4830	 
iterations: 4831	 
iterations: 4832	 
iterations: 4833	 
iterations: 4834	 
iterations: 4835	 
iterations: 4836	 
iterations: 4837	 
iterations: 4838	 
iterations: 4839	 
iterations: 4840	 
iterations: 4841	 
iterations: 4842	 
iterations: 4843	 
iterations: 4844	 
iterations: 4845	 
iterations: 4846	 
iterations: 4847	 
iterations: 4848	 
iterations: 4849	 
iterations: 4850	 
iterations: 4851	 
iterations: 4852	 
iterations: 4853	 
iterations: 4854	 
iterations: 4855	 
iterations: 4856	 
iterations: 4857	 
iterations: 4858	 
iterations: 4859	 
iterations: 4860	 
iterations: 4861	 
iterations: 4862	 
iterations: 4863	 
iterations: 

/home/yifei/miniconda3/envs/BOM/lib/python3.7/site-packages/openpyxl/workbook/child.py:102: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


saving: X Raw Materials
saving: Y Fasteners
saving: Z Vehicle Top Level Cfg

After Sync
EBOM shape:  (4671, 14)
SCBOM shape:  (4994, 78)
SCBOM_updated  (5044, 78)

# of new parts added:  50
# of old parts removed:  373
# of additional columns added in EBOM:  0

This script took--- 839.53 seconds ---
